# Imports

In [1]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

from time import strftime

from numpy.random import seed
seed(888)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [2]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'
NR_CLASSES = 10
IMG_WIDTH = 28
IMG_HEIGHT = 28
IMG_PIXELS = IMG_HEIGHT*IMG_WIDTH
COLOR_CHANNELS = 1
TOTAL_INPUTS = IMG_PIXELS*COLOR_CHANNELS

VALIDATION_SIZE =  10000

# Get the Data

In [3]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 0 ns
Wall time: 11 ms


In [4]:
y_train_all.shape

(60000,)

In [5]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [6]:
y_test.shape

(10000,)

In [7]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_train_all.shape

CPU times: total: 4.27 s
Wall time: 6.73 s


(60000, 784)

In [8]:
%%time
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)
x_test.shape

CPU times: total: 750 ms
Wall time: 1.14 s


(10000, 784)

# Explore and Preprocessing


In [9]:
#Re-Scale
x_train_all, x_test = x_train_all/255.0, x_test/255.0

#### One-Hot Encoding


In [10]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_train_all.shape

(60000, 10)

In [11]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

#### Create Validation Dataset from training data

In [12]:
x_train = x_train_all[VALIDATION_SIZE:]
x_val = x_train_all[:VALIDATION_SIZE]
y_train = y_train_all[VALIDATION_SIZE:]
y_val = y_train_all[:VALIDATION_SIZE]

# Setup Tensorflow Graph

In [71]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='Labels')

## Neural Network Architecture

### Hyperparameters

In [72]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [73]:
# with tf.name_scope('hidden_layer_1'):
#     initial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1], stddev=0.1, seed=42)
#     w1 = tf.Variable(initial_value=initial_w1,name= 'w1')
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1,name= 'b2')
#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out = tf.nn.relu(layer1_in)

In [74]:
# with tf.name_scope('hidden_layer_2'):
#     initial_w2 = tf.truncated_normal(shape=[n_hidden1,n_hidden2], stddev=0.1, seed=42)
#     w2 = tf.Variable(initial_value=initial_w2,name= 'w2')

#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2,name= 'b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)

In [75]:
# with tf.name_scope('output_layer'):
    
#     initial_w3 = tf.truncated_normal(shape=[n_hidden2,NR_CLASSES], stddev=0.1, seed=42)
#     w3 = tf.Variable(initial_value=initial_w3,name= 'w3')

#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3,name= 'b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [76]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w,name= 'W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b,name= 'B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        return layer_out

In [77]:
# model without dropout

# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
#                       bias_dim=[n_hidden1], name='layer_1')
# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                       bias_dim=[n_hidden2], name='layer_2')
# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
#                       bias_dim=[NR_CLASSES], name='out')
# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [78]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')
layer_drop = tf.nn.dropout(layer_1, keep_prob=0.8, name='dropout_layer')
layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

# Tensorboard Setup

In [79]:
folder_name = f'{model_name} at {strftime("%H %M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories')

Successfully created directories


# Loss, Optimisation & Metrics

#### Defining Loss Function

In [80]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining Optimizer

In [81]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

#### Accuracy Metric


In [82]:
with tf.name_scope('accuracy_calc'):
    model_prediction = tf.argmax(output, axis=1, name='prediction')
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [83]:
with tf.name_scope('performance'):
    tf.summary.scalar('acccuracy', accuracy)
    tf.summary.scalar('cost', loss)

#### Check Input Images in Tensorboard

In [84]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

In [85]:
sess = tf.Session()

#### Setup Filewriter and Merge Summaries

In [86]:
merged_summary = tf.summary.merge_all()
train_writer = tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory+'/validation')

#### Initialise all the variables

In [87]:
init = tf.global_variables_initializer()
sess.run(init)

## Batching the Data

In [88]:
size_of_batch = 1000

In [89]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [90]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch = index_in_epoch + batch_size
    
    if index_in_epoch>num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

In [91]:
for epoch in range(nr_epochs):
    #=============================Train===========================
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(size_of_batch, x_train, y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    s,batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    train_writer.add_summary(s, epoch)    
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    #=============================Validation===========================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)
print('Done training!!')

Epoch 0 	| Training Accuracy = 0.8379999995231628
Epoch 1 	| Training Accuracy = 0.8550000190734863
Epoch 2 	| Training Accuracy = 0.8659999966621399
Epoch 3 	| Training Accuracy = 0.8669999837875366
Epoch 4 	| Training Accuracy = 0.8759999871253967
Epoch 5 	| Training Accuracy = 0.9729999899864197
Epoch 6 	| Training Accuracy = 0.9760000109672546
Epoch 7 	| Training Accuracy = 0.9800000190734863
Epoch 8 	| Training Accuracy = 0.9829999804496765
Epoch 9 	| Training Accuracy = 0.9850000143051147
Epoch 10 	| Training Accuracy = 0.9869999885559082
Epoch 11 	| Training Accuracy = 0.9850000143051147
Epoch 12 	| Training Accuracy = 0.9879999756813049
Epoch 13 	| Training Accuracy = 0.9890000224113464
Epoch 14 	| Training Accuracy = 0.9890000224113464
Epoch 15 	| Training Accuracy = 0.9879999756813049
Epoch 16 	| Training Accuracy = 0.9879999756813049
Epoch 17 	| Training Accuracy = 0.9869999885559082
Epoch 18 	| Training Accuracy = 0.9869999885559082
Epoch 19 	| Training Accuracy = 0.9869999

## Saving the Model

In [92]:
outputs = {'accuracy_calc/prediction': model_prediction}
inputs = {'X':X}
tf.compat.v1.saved_model.simple_save(sess, 'SavedModel', inputs, outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


## Make a Prediction

In [93]:
img = Image.open('MNIST/test_img.png')

In [94]:
bw = img.convert('L')
bw

In [95]:
img_array = np.invert(bw)
img_array.shape

(28, 28)

In [96]:
test_img = img_array.ravel()
test_img.shape

(784,)

In [97]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), 
                      feed_dict={X:[test_img]})

In [98]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


## Testing and Evaluation

In [102]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test}) 
print(f'Accuracy for test data set is {test_accuracy:0.2%}')

RuntimeError: Attempted to use a closed Session.

## Reset for the Next Run

In [101]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()